In [1]:
import numpy as np
import tensorflow as tf
import os
from random import shuffle
#import matplotlib.pyplot as plt

In [2]:
#Directories
s_dir = "/home/vishnu/TFlow_ws/CameraProj/MLData/Smile"
o_dir = "/home/vishnu/TFlow_ws/CameraProj/MLData/Other"
#s_dir = r"C:\Users\Vishnu\Desktop\MLData\Smile"
#o_dir = r"C:\Users\Vishnu\Desktop\MLData\Other"

#Log dir
LOGDIR = "/home/vishnu/TFlow_ws/CameraProj/src/logs"
#LOGDIR = r"C:\Users\Vishnu\Desktop\MLData\logs"

In [3]:
def get_files(Sdir,Odir):
    filelist = [Sdir+"/"+x for x in os.listdir(Sdir)]+[Odir+"/"+y for y in os.listdir(Odir)]
    shuffle(filelist)
    label = [1 if x.find('Smile') != -1 else 0 for x in filelist]
    return filelist, label
    
def data_aug(filenames,labels):
    img_string = tf.read_file(filenames,name="data_aug_read_files")
    img_decode = tf.image.decode_jpeg(img_string,channels=1)
    img = tf.cast(img_decode,tf.float32,name="cast_float_32")
    img = tf.image.per_image_standardization(img)
    
    mirror_img = tf.image.flip_left_right(img)
    one_hot = tf.one_hot(labels,2)
    
    
    return [img,mirror_img],one_hot

In [4]:
#Setting up data for model
filenames,labels = get_files(s_dir,o_dir)
dataset_aug = tf.data.Dataset.from_tensor_slices((filenames,labels))
dataset_aug = dataset_aug.map(data_aug)

it = tf.data.Iterator.from_structure(dataset_aug.output_types,dataset_aug.output_shapes)
it_op = it.make_initializer(dataset_aug)

x,y_ = it.get_next()
#Variable initialize operation
init_op = tf.global_variables_initializer()

In [5]:
#Parsing Data
data_array = []

with tf.Session() as sess:
    sess.run(init_op)
    sess.run(it_op)
    
    while True:
        try:
            rtn,yy = sess.run([x,y_])
            data_array.append([rtn[0],yy])
            data_array.append([rtn[1],yy])
            
        except tf.errors.OutOfRangeError:
            print("Finished")
            break
shuffle(data_array)

#Splitting data into train and other
length = len(data_array)
third = 2*int(length/3.0)

data_array = np.array(data_array)
Train_data = data_array[:third][:,0]
Train_data_labels = data_array[:third][:,1]
Test_data = data_array[third:][:,0]
Test_data_labels = data_array[third:][:,1]

count = 0;
for i in range(len(Train_data_labels)):
    if (Train_data_labels[i][0] == 1):
        count = count+1

print(len(Train_data_labels),count)

Finished
(2660, 1325)


In [6]:
#Tflow functions

def conv_layer(m_input,size_in,size_out,k_size_w,k_size_h,pool_k_size,pool_stride_size,name,num):
    with tf.name_scope(name+num):
        w = tf.Variable(tf.truncated_normal([k_size_w,k_size_h,size_in,size_out],stddev = 0.1),name=("w"+num))
        b = tf.Variable(tf.constant(0.1,shape=[size_out]),name=("b"+num))
        
        conv = tf.nn.conv2d(m_input,w,strides=[1,1,1,1],padding="SAME")
        act = tf.nn.leaky_relu((conv+b),alpha=0.3)
        #.3
        
        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)
        return tf.nn.max_pool(act,ksize=[1,pool_k_size,pool_k_size,1],strides=[1,pool_stride_size,pool_stride_size,1],padding="SAME")
    
def fc_layer(m_input,size_in,size_out,name,num):
    with tf.name_scope(name+num):
        w = tf.Variable(tf.truncated_normal([size_in,size_out],stddev=0.1),name=("w"+num))
        b = tf.Variable(tf.constant(0.1,shape=[size_out]),name=("b"+num))
        act = tf.nn.leaky_relu(tf.matmul(m_input,w)+b,alpha=0.3)

        tf.summary.histogram("weights",w)
        tf.summary.histogram("biases",b)
        tf.summary.histogram("activations",act)
        return act

In [7]:
#model function

def model(start_learning_rate,lr_decay,conv_count,fc_count,conv_feats,fc_feats,hparam):
    global LOGDIR
    global Train_data
    if (len(conv_feats) != conv_count):
        return
    
    tf.reset_default_graph()
    
    with tf.name_scope("place_holders"):
        x = tf.placeholder(tf.float32,shape=[50,50,1],name="x")
        x_image = tf.reshape(x,[-1,50,50,1])
        y = tf.placeholder(tf.float32,shape=[None,2],name="y")
    
    with tf.name_scope("variable"):
        learning_rt = tf.Variable(start_learning_rate,name="learning_rt")
        global_step = tf.Variable(0,trainable=False)
        tf.summary.scalar("Learning_rate",learning_rt)
        
    with tf.name_scope("lr_decay"):
        dec_learning_rate = tf.train.exponential_decay(learning_rt,global_step,len(Train_data),lr_decay,staircase=True)
        
        
    
    tf.summary.image("image_input",x_image,1)
    
    convs = []
    convs.append(x_image)
    conv_name = "conv"
    for i in range(0,conv_count-1):
        convs.append(conv_layer(convs[i],conv_feats[i],conv_feats[i+1],5,5,2,2,"conv",str(i+1)))
    #print((conv_layer(convs[0],1,conv_feats[0],5,5,2,2,"conv",str(0+1)).shape))
        
    
    shape = (convs[conv_count-1]).get_shape().as_list()
    fc_feats[0] = shape[1]*shape[2]*conv_feats[conv_count-1]
    flatten = tf.reshape(convs[conv_count-1],[1,fc_feats[0]])
    
    fcs = []
    fcs.append(flatten)
    fcs_name = "FC"
    for i in range(0,fc_count-1):
        fcs.append((fc_layer(fcs[i],fc_feats[i],fc_feats[i+1],"FC",str(i+1))))
    
    
    logts = fcs[len(fcs)-1]

    with tf.name_scope("cross_entropy"):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logts,labels=y),name="cross_entropy")
        tf.summary.scalar("cross_entropy",cross_entropy)

    with tf.name_scope("Train"):
        train_step = tf.train.AdamOptimizer(dec_learning_rate).minimize(cross_entropy)
    
    with tf.name_scope("accuracy"):
        pred = tf.equal(tf.argmax(logts,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
        tf.summary.scalar("accuracy",accuracy)
    
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR+hparam)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        cor = 0
        for i in range(len(Train_data)):
            l = Train_data_labels[i].reshape(1,2)
        
            if i % 5 == 0:
                [train_accuracy,s] = sess.run([accuracy,summ],feed_dict={x: Train_data[i],y: l})
                writer.add_summary(s,i)
        
            a = sess.run([train_step],feed_dict={x: Train_data[i],y: l})
    
        for i in range(len(Test_data)):
            l = Test_data_labels[i].reshape(1,2)
            cor_pred = sess.run(pred,feed_dict={x: Test_data[i],y: l})
            if (cor_pred):
                cor = cor+1
        
        writer.add_graph(sess.graph)   
        writer.close()

        print(float(cor)/len(Test_data))

In [8]:
#Feats that worked
#75
#rt = 1e-4
#conv_feats=[1,16,32,64]->between 8-16
#fc_feats=[0,1500,500,50,2] -> Too many neurons causing model not to train at all
#77%
#rt = 1e-4
#conv_feats=[1,8,16,32]
#fc_feats=[0,1000,500,50,2]
#77%
#rt = 1e-4
#conv_feats=[1,8,12,16]
#fc_feats=[0,700,300,100,2]
#77%
#rt = 1e-4
#conv_feats=[1,8,10,12]
#fc_feats=[0,700,300,150,2]
#79%
#rt = 1e-4
#conv_feats=[1,8,10,12]
#fc_feats=[0,300,150,50,2]
#78%
#conv_feats=[1,3,8,12]
#fc_feats=[0,100,100,100,2]
#82% lnrt=1e-3 dec = .6
#conv_feats=[1,8,10,12]
#fc_feats=[0,150,50,2]
len_rt= 1e-3
conv_count = 4
fc_count = 4
conv_feats=[1,8,10,12]
#fc_feats=[0,300,150,50,2]
fc_feats=[0,150,50,2]
model(len_rt,.6,conv_count,fc_count,conv_feats,fc_feats,"test1")

0.795795795796
